# Current State of Hanover (ZCTA 21076)

A Hanover-centric, reproducible data analysis using only official datasets. Media references are included for narrative context only (no numeric extraction).

- Geography: ZCTA 21076 (Hanover, MD) as the primary unit
- Comparisons: Howard County, Anne Arundel County, Maryland, United States (as available)
- Time horizon: ACS 2019–2023 5-year estimates for ZCTA-level; 2024–2025 MD labor context
- Reproducibility: All sources archived under `data/raw/` with URLs and retrieval timestamps; processed artifacts under `data/processed/`
- Outputs: Figures in `output/figures/`, maps in `output/maps/`, and exported HTML report
- Guardrails: No inferences; all quantitative claims trace to archived sources with methods and uncertainty noted

In [1]:
# 1) Set Up Environment and Reproducibility
%pip -q install pandas numpy geopandas shapely requests seaborn matplotlib statsmodels scikit-learn pandera folium

import os, sys, platform, json, time, hashlib, random
from datetime import datetime, timezone
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Optional: geopandas & folium
import geopandas as gpd
from shapely.geometry import shape, Polygon, mapping
import folium

# Styling and reproducibility
np.random.seed(21076)
random.seed(21076)
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')

# Project paths
ROOT = os.path.abspath(os.path.join(os.getcwd()))
DATA_RAW = os.path.join(ROOT, 'data', 'raw')
DATA_PROCESSED = os.path.join(ROOT, 'data', 'processed')
OUTPUT_FIG = os.path.join(ROOT, 'output', 'figures')
OUTPUT_MAP = os.path.join(ROOT, 'output', 'maps')
for d in (DATA_RAW, DATA_PROCESSED, OUTPUT_FIG, OUTPUT_MAP):
    os.makedirs(d, exist_ok=True)

# Log environment metadata
ENV_META = {
    'timestamp_utc': datetime.now(timezone.utc).isoformat(),
    'python': sys.version,
    'platform': platform.platform(),
    'cwd': os.getcwd(),
}
print(json.dumps(ENV_META, indent=2))


Note: you may need to restart the kernel to use updated packages.
{
  "timestamp_utc": "2025-09-22T04:32:32.586373+00:00",
  "python": "3.13.7 (main, Aug 14 2025, 11:12:11) [Clang 17.0.0 (clang-1700.3.19.1)]",
  "platform": "macOS-26.0-arm64-arm-64bit-Mach-O",
  "cwd": "/Users/stan/Library/Mobile Documents/com~apple~CloudDocs/Developer/MarylandData/analysis"
}
{
  "timestamp_utc": "2025-09-22T04:32:32.586373+00:00",
  "python": "3.13.7 (main, Aug 14 2025, 11:12:11) [Clang 17.0.0 (clang-1700.3.19.1)]",
  "platform": "macOS-26.0-arm64-arm-64bit-Mach-O",
  "cwd": "/Users/stan/Library/Mobile Documents/com~apple~CloudDocs/Developer/MarylandData/analysis"
}


## 2) Define Primary Geography: Hanover Boundary and Metadata

- Primary unit: ZCTA 21076
- Target CRS: EPSG:4326 (WGS84) for storage; EPSG:26985 (NAD83 / Maryland) for area in meters → convert to km²
- Output: `data/processed/hanover_boundary.geojson` with centroid and area stats saved to `data/processed/hanover_boundary_stats.json`
- Provenance: Persist full TIGERweb request URL and retrieval timestamp in stats

Next, we will also fetch:
- ACS 2023 median gross rent (B25064_001E) for Hanover affordability context
- 2020 Decennial PL population (P1_001N) to compute 2020→2023 growth using actual Census counts, not estimates.

In [2]:
# Fetch ZCTA 21076 boundary via TIGER/ACS crosswalk (using Census TIGER Zip Code Tabulation Areas)
# We avoid external calls if cached; persist to data/processed/ with CRS handling and provenance.

import os, json
from datetime import datetime, timezone
import requests
from urllib.parse import urlencode
import geopandas as gpd
from shapely.ops import unary_union

ZCTA = '21076'

boundary_path = os.path.join(DATA_PROCESSED, 'hanover_boundary.geojson')
stats_path = os.path.join(DATA_PROCESSED, 'hanover_boundary_stats.json')

if not os.path.exists(boundary_path):
    # Use TIGERweb ZCTA sublayer (ID 11) under PUMA_TAD_TAZ_UGA_ZCTA service
    arc_url = (
        'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/PUMA_TAD_TAZ_UGA_ZCTA/MapServer/11/query'
    )
    params = {
        'where': f"ZCTA5 = '{ZCTA}'",
        'outFields': '*',
        'f': 'geojson',
        'outSR': 4326,
        'returnGeometry': 'true',
    }
    r = requests.get(arc_url, params=params, timeout=60)
    r.raise_for_status()
    gj = r.json()
    if 'features' not in gj or len(gj['features']) == 0:
        raise RuntimeError('No ZCTA features returned from TIGERweb for 21076')
    with open(boundary_path, 'w') as f:
        json.dump(gj, f)
    request_url = r.url
    fetched = True
else:
    with open(boundary_path, 'r') as f:
        gj = json.load(f)
    request_url = None
    fetched = False

# Build GeoDataFrame
hanover_gdf = gpd.GeoDataFrame.from_features(gj['features'], crs='EPSG:4326')
# Project for area calcs (NAD83 / Maryland meters)
hanover_proj = hanover_gdf.to_crs('EPSG:26985')
# Area in km^2 from m^2
area_sqkm = float(hanover_proj.geometry.area.sum() / 1_000_000.0)
# Robust centroid using unary_union, then convert to WGS84
centroid_proj = unary_union(hanover_proj.geometry).centroid
centroid_wgs84 = gpd.GeoSeries([centroid_proj], crs='EPSG:26985').to_crs('EPSG:4326').iloc[0]

stats = {
    'zcta': ZCTA,
    'crs': 'EPSG:4326',
    'projected_crs': 'EPSG:26985',
    'area_sq_km': area_sqkm,
    'centroid_lon': centroid_wgs84.x,
    'centroid_lat': centroid_wgs84.y,
    'source': 'Census TIGERweb ZCTA (PUMA_TAD_TAZ_UGA_ZCTA/11)',
    'tigerweb_request_url': request_url,
    'retrieved_at': datetime.now(timezone.utc).isoformat(),
    'fetched_this_run': fetched,
}
with open(stats_path, 'w') as f:
    json.dump(stats, f, indent=2)

hanover_gdf.head(1)

,geometry,AREALAND,AREAWATER,BASENAME,CENTLAT,CENTLON,FUNCSTAT,GEOID,HU100,INTPTLAT,INTPTLON,LSADC,MTFCC,NAME,OBJECTID,OID,POP100,ZCTA5,ZCTA5CC
0,"POLYGON ((-76.7671 39.16238, -76.76682 39.1622...",34039826,19407,21076,+39.1685435,-076.7223887,S,21076,9441,+39.1714019,-076.7233409,Z5,G6350,ZCTA5 21076,22704,221704257715385,22299,21076,B5


## 3) ACS fetch helpers with caching and provenance
We'll create tiny helpers to request ACS 5-year data for ZCTA 21076 (and later block groups), persist the raw JSON with a timestamp, and return tidy DataFrames with margins of error where available.

In [3]:
import time
from pathlib import Path

CENSUS_BASE = 'https://api.census.gov/data/2023/acs/acs5'
ZCTA_GEO = 'zip code tabulation area:21076'

RAW_DIR = Path(DATA_RAW)
PROC_DIR = Path(DATA_PROCESSED)
RAW_DIR.mkdir(parents=True, exist_ok=True)
PROC_DIR.mkdir(parents=True, exist_ok=True)


def acs_get(variables, base=CENSUS_BASE, geo=ZCTA_GEO, api_key=os.getenv('CENSUS_API_KEY'), note=None):
    """
    Contract:
    - inputs: variables (list[str]), base URL, geo string, optional API key
    - output: dict with 'columns', 'rows', 'url', 'timestamp_utc'
    - error modes: raises on HTTP or schema mismatch; retries 3x with backoff
    """
    assert isinstance(variables, (list, tuple)) and variables, 'variables required'
    params = {
        'get': ','.join(variables),
        'for': geo,
    }
    if api_key:
        params['key'] = api_key
    url = base + '?' + urlencode(params)

    # retry
    last_exc = None
    for attempt in range(3):
        try:
            r = requests.get(base, params=params, timeout=60)
            r.raise_for_status()
            data = r.json()
            if not isinstance(data, list) or len(data) < 2:
                raise RuntimeError('Unexpected ACS response shape')
            out = {
                'columns': data[0],
                'rows': data[1:],
                'url': r.url,
                'timestamp_utc': datetime.now(timezone.utc).isoformat(),
                'note': note,
            }
            # persist raw
            raw_name = f"acs5_2023_{variables[0]}_{int(time.time())}.json"
            with open(RAW_DIR / raw_name, 'w') as f:
                json.dump(out, f, indent=2)
            return out
        except Exception as e:
            last_exc = e
            time.sleep(1 + attempt)
    raise last_exc


def acs_to_df(payload):
    cols = payload['columns']
    df = pd.DataFrame(payload['rows'], columns=cols)
    # coerce numerics where possible
    for c in df.columns:
        if c.endswith('E') or c.endswith('M'):
            df[c] = pd.to_numeric(df[c], errors='coerce')
    return df

# Minimal smoke test: total population and median household income
VARS_DEMO = ['B01003_001E', 'B19013_001E']
raw = acs_get(VARS_DEMO, note='smoke-test ZCTA 21076')
df_demo = acs_to_df(raw)
df_demo

,B01003_001E,B19013_001E,zip code tabulation area
0,23642,143409,21076


In [4]:
# Fetch ACS median gross rent and Decennial population with provenance
VARS_RENT = ['B25064_001E']
rent_raw = acs_get(VARS_RENT, note='median gross rent ZCTA 21076')
rent_df = acs_to_df(rent_raw)
print('ACS rent request URL:', rent_raw['url'])

# Fetch 2020 Decennial DHC P1_001N for growth baseline (DHC supports ZCTA; PL does not)
import requests
from urllib.parse import urlencode

DEC_BASE = 'https://api.census.gov/data/2020/dec/dhc'
params_dec = {
    'get': 'P1_001N',
    'for': 'zip code tabulation area:21076',
}
if os.getenv('CENSUS_API_KEY'):
    params_dec['key'] = os.getenv('CENSUS_API_KEY')

r_dec = requests.get(DEC_BASE, params=params_dec, timeout=60)
r_dec.raise_for_status()
dec_data = r_dec.json()
assert isinstance(dec_data, list) and len(dec_data) >= 2, 'Unexpected Decennial response'

# Persist raw
raw_name_dec = f"decennial_2020_dhc_P1_001N_{int(time.time())}.json"
with open(RAW_DIR / raw_name_dec, 'w') as f:
    json.dump({'columns': dec_data[0], 'rows': dec_data[1:], 'url': r_dec.url, 'timestamp_utc': datetime.now(timezone.utc).isoformat()}, f, indent=2)

# Convert to DataFrame
import pandas as pd
dec_df = pd.DataFrame(dec_data[1:], columns=dec_data[0])
rent_df, dec_df

ACS rent request URL: https://api.census.gov/data/2023/acs/acs5?get=B25064_001E&for=zip+code+tabulation+area%3A21076&key=49c7cb8a64fa2d5d754e347a492be7c5067e9575


(   B25064_001E zip code tabulation area
 0         2337                    21076,
   P1_001N zip code tabulation area
 0   22299                    21076)

## 4) ACS key variables: add median gross rent (B25064_001E) and Decennial baseline
We extend our ACS helper usage to include median gross rent for ZCTA 21076 and also fetch 2020 Decennial PL (P1_001N) for a clean growth baseline. All raw payloads and request URLs are persisted under `data/raw/` with timestamps for reproducibility.

In [5]:
import pandera as pa
from pandera import Column, DataFrameSchema

# Define schemas for boundary stats and ACS demo output
# Relax boundary validation: check for essential identifiers when present, not strict TIGER field names
boundary_required_any = ['ZCTA5', 'GEOID', 'GEOID10', 'ZCTA5CE10']
existing_cols = set(hanover_gdf.columns)
required_present = any(col in existing_cols for col in boundary_required_any)
if not required_present:
    raise ValueError(f"Boundary lacks expected identifier columns: any of {boundary_required_any}")

# Keep ACS schema strict
acs_demo_schema = DataFrameSchema({
    'B01003_001E': Column(float, nullable=True, coerce=True),
    'B19013_001E': Column(float, nullable=True, coerce=True),
    'zip code tabulation area': Column(str, nullable=False),
})

# Validate ACS (coerce to float for estimates)
_ = acs_demo_schema.validate(df_demo)
print('Schema validated: ACS demo; boundary identifiers present:', required_present)

Schema validated: ACS demo; boundary identifiers present: True


/Users/stan/Library/Mobile Documents/com~apple~CloudDocs/Developer/MarylandData/.venv/lib/python3.13/site-packages/pandera/_pandas_deprecated.py:149: FutureWarning: Importing pandas-specific classes and functions from the
top-level pandera module will be **removed in a future version of pandera**.
If you're using pandera to validate pandas objects, we highly recommend updating
your import:

```
# old import
import pandera as pa

# new import
import pandera.pandas as pa
```

If you're using pandera to validate objects from other compatible libraries
like pyspark or polars, see the supported libraries section of the documentation
for more information on how to import pandera:

https://pandera.readthedocs.io/en/stable/supported_libraries.html

To disable this warning, set the environment variable:

```
export DISABLE_PANDERA_IMPORT_WARNING=True
```

  warnings.warn(_future_warning, FutureWarning)


In [6]:
# Path correction: ensure we write under repo-root data/processed, not analysis/data/processed
from pathlib import Path

def find_repo_root(start: Path) -> Path:
    cur = start
    for _ in range(5):  # climb a few levels at most
        if (cur / 'README.md').exists() and (cur / 'data').exists():
            return cur
        cur = cur.parent
    return start

repo_root = find_repo_root(Path.cwd())
correct_proc = repo_root / 'data' / 'processed'
print('DATA_PROCESSED current:', DATA_PROCESSED)
print('Repo-root processed:', correct_proc)

# If current DATA_PROCESSED points inside analysis/, switch to root-level path
if 'analysis/data/processed' in str(DATA_PROCESSED) or Path(DATA_PROCESSED).resolve() != correct_proc.resolve():
    DATA_PROCESSED = str(correct_proc)
    os.makedirs(DATA_PROCESSED, exist_ok=True)
    # Move boundary artifacts if they exist under analysis/data/processed
    src_dir = Path.cwd() / 'data' / 'processed'
    moved = []
    for name in ('hanover_boundary.geojson', 'hanover_boundary_stats.json'):
        src = src_dir / name
        dst = correct_proc / name
        if src.exists() and not dst.exists():
            src.replace(dst)
            moved.append(name)
    print('Repointed DATA_PROCESSED to', DATA_PROCESSED, 'moved:', moved)
else:
    print('DATA_PROCESSED already at repo-root path.')

DATA_PROCESSED current: /Users/stan/Library/Mobile Documents/com~apple~CloudDocs/Developer/MarylandData/analysis/data/processed
Repo-root processed: /Users/stan/Library/Mobile Documents/com~apple~CloudDocs/Developer/MarylandData/data/processed
Repointed DATA_PROCESSED to /Users/stan/Library/Mobile Documents/com~apple~CloudDocs/Developer/MarylandData/data/processed moved: []


In [7]:
# Persist MDOT/Open Data catalog seeds for reproducibility
from datetime import datetime, timezone
import json, os

mdot_catalog = {
    'timestamp_utc': datetime.now(timezone.utc).isoformat(),
    'note': 'Seed datasets relevant to Maryland transportation context; provenance only, no numeric use here.',
    'datasets': [
        {
            'id': '8hpm-qsj3',
            'title': 'MDOT Performance Dashboard - Annual Data',
            'portal': 'opendata.maryland.gov',
            'link': 'https://opendata.maryland.gov/d/8hpm-qsj3'
        },
        {
            'id': 'exua-btti',
            'title': 'Maryland Annual Vehicle Miles of Travel',
            'portal': 'opendata.maryland.gov',
            'link': 'https://opendata.maryland.gov/d/exua-btti'
        },
        {
            'id': 'amgn-fptx',
            'title': 'SHA Districts by County',
            'portal': 'opendata.maryland.gov',
            'link': 'https://opendata.maryland.gov/d/amgn-fptx'
        }
    ]
}
seed_path = os.path.join(DATA_RAW, 'mdot_catalog_seeds.json')
with open(seed_path, 'w') as f:
    json.dump(mdot_catalog, f, indent=2)
print('Wrote catalog seeds to', seed_path)

Wrote catalog seeds to /Users/stan/Library/Mobile Documents/com~apple~CloudDocs/Developer/MarylandData/analysis/data/raw/mdot_catalog_seeds.json


In [8]:
# Validate boundary stats provenance keys exist
import json, os
stats_path = os.path.join(DATA_PROCESSED, 'hanover_boundary_stats.json')
with open(stats_path, 'r') as f:
    stats_loaded = json.load(f)
req_keys = ['tigerweb_request_url', 'retrieved_at', 'fetched_this_run', 'area_sq_km', 'centroid_lon', 'centroid_lat']
missing = [k for k in req_keys if k not in stats_loaded]
print('Boundary stats keys present:', sorted(set(stats_loaded.keys()))[:8], '...')
assert not missing, f"Missing provenance keys in stats: {missing}"

Boundary stats keys present: ['area_sq_km', 'centroid_lat', 'centroid_lon', 'crs', 'fetched_this_run', 'projected_crs', 'retrieved_at', 'source'] ...
